# NavSim Car Demo

This notebook tests the airsim car simulations and trains for 2 episodes, 100 timesteps each episode.

In the following cell make the changes to the paths as needed.

In the third cell from here, make changes to the RL Config or Environment Config as needed.

In [1]:
from pathlib import Path

# Please change the following to the base path to the repos. All our code repos were in d:/work/projects. 

#base_path=Path("d:/")
#base_path = base_path / "work" / "projects"

base_path = Path.home() / "projects"


# Change the following path to your Unity Env Executable
# if you want to use Unity Editor then uncomment the last line which sets uenv_file to None

# uenv_path = Path("d:/") / 'work' 
uenv_path = Path.home() 
uenv_path = uenv_path / 'unity-envs' / "Berlin_URP" / 'Berlin_ML'
uenv_file = str(uenv_path.resolve())

#uenv_file = None  

In [2]:
import sys

%reload_ext autoreload
%autoreload 2

for pkg in ['ezai_util','ai_coop_py']:
    pkg_path = base_path / pkg
    pkg_path = str(pkg_path.resolve())
    print(pkg_path)
    if not pkg_path in sys.path:
        sys.path.append(pkg_path)
import ezai_util
import navsim
from ezai_util import DictObj

/mnt/data/afandang/projects/ezai_util
/mnt/data/afandang/projects/ai_coop_py
/mnt/data/afandang/projects/ezai_util


Set the configuration either from the cell below or from the config file.

Set the observation mode to 0,1,2 

In [3]:
conf = {
  "env_conf": {
    "filename": "/home/afandang/unity-envs/Berlin_ML/Berlin_ML",
    "log_folder": "unity.log",
    "seed": 123,
    "timeout": 600,
    "worker_id": 0,
    "base_port": 5005,
    "observation_mode": 0,
    "max_steps": 100,
    "reward_for_goal": 50,
    "reward_for_ep": 0.005,
    "reward_for_other": -0.1,
    "reward_for_falling_off_map": -50,
    "reward_for_step": -0.0001,
    "segmentation_mode": 1
  },
  "run_conf": {
    "env_name": "navsim",
    "episode_max_steps": 100,
    "num_episodes": 2,
    "memory_capacity": 10000,
    "seed": 123,
    "discount": 0.99,
    "tau": 5e-3,
    "expl_noise": 0.1,
    "batch_size": 256,
    "batches_before_train": 2,
    "checkpoint_interval": 1
  }
}

In [24]:
conf = DictObj().load_from_json_file("navsim_conf.json")

ezai - I - Making empty DictObj because parameters passed is not a dict
2020-12-09 16:22:48 INFO [dict.py:54] Making empty DictObj because parameters passed is not a dict


In [25]:
# you can also change the observation mod or any other variable after reading the file
conf.env_conf["observation_mode"]=2

In [26]:
print(conf)

{
    "env_conf": {
        "filename": "/home/afandang/unity-envs/Berlin_ML/Berlin_ML",
        "log_folder": "unity.log",
        "seed": 123,
        "timeout": 600,
        "worker_id": 0,
        "base_port": 5005,
        "observation_mode": 2,
        "max_steps": 100,
        "reward_for_goal": 50,
        "reward_for_ep": 0.005,
        "reward_for_other": -0.1,
        "reward_for_falling_off_map": -50,
        "reward_for_step": -0.0001,
        "segmentation_mode": 1
    },
    "run_conf": {
        "env_name": "navsim",
        "episode_max_steps": 100,
        "num_episodes": 2,
        "memory_capacity": 10000,
        "seed": 123,
        "discount": 0.99,
        "tau": 0.005,
        "expl_noise": 0.1,
        "batch_size": 256,
        "batches_before_train": 2,
        "checkpoint_interval": 1
    }
}


In [7]:
trainer = navsim.Trainer(run_id='navsim_demo',run_resume=False,conf=conf)

2020-12-09 11:25:26 INFO [environment.py:108] Connected to Unity environment with package version 1.1.0-preview and communication version 1.0.0
2020-12-09 11:25:35 INFO [environment.py:265] Connected new brain:
VectorVisualNavigator?team=0


Unity env creation resource usage: 
time:0.057497782,peak_memory:2258923,current_memory:2102982

Env Info
-----------
Action Space: Box(-1.0, 1.0, (2,), float32)
Action sample: [ 0.39293838 -0.42772132]
Action Space Shape: (2,)
Action Space Low: [-1. -1.]
Action Space High: [1. 1.]
Observation Mode: 2
Gym Observation Space: Tuple(Box(0.0, 1.0, (84, 84, 3), float32), Box(0.0, 1.0, (84, 84, 1), float32), Box(0.0, 1.0, (84, 84, 3), float32), Box(-inf, inf, (10,), float32))
Gym Observation Space Shape: None
Self Observation Space: Tuple(Box(0.0, 1.0, (84, 84, 3), float32), Box(0.0, 1.0, (84, 84, 1), float32), Box(0.0, 1.0, (84, 84, 3), float32), Box(-inf, inf, (10,), float32))
Self Observation Space Shape: None
Self Observation Space Shapes: [(84, 84, 3), (84, 84, 1), (84, 84, 3), (10,)]
Reward Range: (-inf, inf)
Metadata: {'render.modes': ['rgb_array']}


In [28]:
trainer.agent.save_to_onnx()

In [ ]:
import netron

In [21]:
netron.start('actor.onnx')

Serving 'actor.onnx' at http://localhost:8080


In [22]:
netron.start('critic.onnx')


Stopping http://localhost:8080
Serving 'critic.onnx' at http://localhost:8080


In [23]:
trainer.train()

100%|██████████| 2/2 [00:39<00:00, 19.69s/it]


In [ ]:
trainer.agent.save_actor('model.onnx')

In [29]:
trainer.env_close()
trainer.files_close()

2020-12-09 16:28:38 INFO [environment.py:417] Environment shut down with return code 0.
